In [1]:
%reload_ext autoreload
%autoreload 2   

import sys
sys.path.append('../src_jobs/')

In [2]:
import torch
import warnings

# stop warnings
torch.set_float32_matmul_precision("high")
warnings.filterwarnings("ignore", ".*does not have many workers.*")

/opt/miniconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

#autheticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient.from_config(
    credential=credential,
    path="config.json",
)

Found the config file in: config.json


In [4]:
# specify aml compute name.
cpu_compute_target = "cpu-cluster"
gpu_compute_target = "gpu-cluster"

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print("CPU compute target not found.")
    
try:
    ml_client.compute.get(gpu_compute_target)
except Exception:
    print("GPU compute target not found.")

In [5]:
from azure.ai.ml.entities import Environment

myenv = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.2-cudnn8-ubuntu18.04",
    conda_file="/workspaces/AICoE_Ramping_Artefacts/conda.yml",
    name="Pytorch112_python310_cuda10_cudnn8_ubuntu18",
    description="Environment created from a Docker image plus Conda environment, based on RGB towers files, using a base image including cuda.",
)
ml_client.environments.create_or_update(myenv)

# curated_env_name = "acpt-pytorch-2.0-cuda11.7@latest"


Environment({'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'Pytorch112_python310_cuda10_cudnn8_ubuntu18', 'description': 'Environment created from a Docker image plus Conda environment, based on RGB towers files, using a base image including cuda.', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': True, 'id': '/subscriptions/8de3e85d-b97f-48c1-a25b-5bddf9dc484c/resourceGroups/m3-mlops-dev/providers/Microsoft.MachineLearningServices/workspaces/m3-mlops-mlw-dev/environments/Pytorch112_python310_cuda10_cudnn8_ubuntu18/versions/4', 'Resource__source_path': None, 'base_path': '/workspaces/AICoE_Ramping_Artefacts/artifactory-master/notebooks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f7362c3b2b0>, 'serialize': <msrest.serialization.Serializer object at 0x7f72880a5e40>, 'version': '4', 'latest_version': None, 'conda_file': {'channels': ['defaults'], 'depe

In [6]:
envs = ml_client.environments.list(name="Pytorch112_python310_cuda10_cudnn8_ubuntu18")
for env in envs:
    print(env.version)

4
3
2
1


In [6]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential

# #autheticate
# credential = DefaultAzureCredential()

# # Get a handle to the workspace
# ml_client = MLClient.from_config(
#     credential=credential,
#     path="config.json",
# )

# Set the version number of the data asset (for example: '1')
VERSION = "2"
path = "/workspaces/AICoE_Ramping_Artefacts/artifactory-master/data/validation512.london.pkl"

# Define the Data asset object
my_data = Data(
    path=path,
    type=AssetTypes.URI_FILE,
    description="just using london smart meters dataset for validation (normed with std & mean)",
    name="artifactory_validation_london",
    version=VERSION,
)

# Create the data asset in the workspace
ml_client.data.create_or_update(my_data)

Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'artifactory_validation_london', 'description': 'just using london smart meters dataset for validation (normed with std & mean)', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/8de3e85d-b97f-48c1-a25b-5bddf9dc484c/resourceGroups/m3-mlops-dev/providers/Microsoft.MachineLearningServices/workspaces/m3-mlops-mlw-dev/data/artifactory_validation_london/versions/2', 'Resource__source_path': None, 'base_path': '/workspaces/AICoE_Ramping_Artefacts/artifactory-master/notebooks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f5783e60550>, 'serialize': <msrest.serialization.Serializer object at 0x7f5783e60b20>, 'version': '2', 'latest_version': None, 'path': 'azureml://subscriptions/8de3e85d-b97f-48c1-a25b-5bddf9dc484c/resourcegroups/m3-mlops-

In [9]:
from azure.ai.ml import command, Input, Output
from azure.ai.ml.constants import AssetTypes, InputOutputModes

# data_asset_input = ml_client.data.get("tsdb_tser_norm", version="1")
data_asset_output = "azureml://datastores/workspaceblobstore/paths/artifactory/april/"
data_asset_input = ml_client.data.get("train_dataset_tser_tsdb_normalized", version="3")
# data_asset_validation = ml_client.data.get("artifactory_validation_slidingWindow", version="1")
# data_asset_validation = ml_client.data.get("artifactory_validation_norm", version="3")
data_asset_validation = ml_client.data.get("artifactory_validation_slidingWindow_nolondon", version="1")
data_asset_model = ml_client.data.get("artifactory_output", version="48")

name = "Train_fineTune_slidingWindow_warmupLR"
# define the command
# TODO: later - add config parameters to input
command_job = command(
    code="/workspaces/AICoE_Ramping_Artefacts/artifactory-master/src_jobs",
    command="python train_fineTune_slidingWindow.py --input-path ${{inputs.tser_tsdb}} --val-path ${{inputs.val_path}} --model-path ${{inputs.model_path}} --output-path ${{outputs.output_path}}",
    environment="Pytorch112_python310_cuda10_cudnn8_ubuntu18@latest",
    inputs={
        "tser_tsdb": Input(
            type=AssetTypes.URI_FOLDER,
            mode=InputOutputModes.RO_MOUNT,
            path=data_asset_input.id,
        ),
        "val_path": Input(
            type=AssetTypes.URI_FILE,
            mode=InputOutputModes.RO_MOUNT,
            path=data_asset_validation.id,
        ),
        "model_path": Input(
            type=AssetTypes.URI_FILE,
            mode=InputOutputModes.RO_MOUNT,
            path=data_asset_model.id,
        )
    },
    outputs={
        "output_path": Output(
            type=AssetTypes.URI_FOLDER,
            mode=InputOutputModes.RW_MOUNT,
            path=data_asset_output, 
            name=f"output_{name}"
        )
    },
    compute=gpu_compute_target,
    display_name=name,
    experiment_name="transformer_fineTune_slidingWindow",
)

In [10]:
# submit the command
returned_job = ml_client.jobs.create_or_update(command_job)
# get a URL for the status of the job
returned_job.studio_url

Uploading src_jobs (0.23 MBs): 100%|██████████| 228787/228787 [00:00<00:00, 257442.50it/s] 




'https://ml.azure.com/runs/sincere_vinegar_rlh17tv75m?wsid=/subscriptions/8de3e85d-b97f-48c1-a25b-5bddf9dc484c/resourcegroups/m3-mlops-dev/workspaces/m3-mlops-mlw-dev&tid=9c8f80a6-c648-4c20-90ff-5d267c8ae785'

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

run_model = Model(
    path="azureml://jobs/{}/outputs/artifacts/paths/model/".format(returned_job.name),
    name="run-model-example",
    description=f"Model created from run {returned_job.name}",
    type=AssetTypes.MLFLOW_MODEL
)

ml_client.models.create_or_update(run_model)